In [238]:
import pandas as pd
from sklearn.decomposition import PCA
import torch
import numpy as np
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [193]:
df_meteo_eto = pd.read_csv("../../DATASETS_TRACTATS/df_meteo_eto.csv")
df_train = pd.read_csv("../../DATASETS_TRACTATS/TOT_menys14_15.csv")

In [194]:
df_train = df_train.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [197]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "ALTITUD", "VARIEDAD", "COLOR", "TIPO", "MODO"])
df_train = encoder.fit_transform(df_train)

In [200]:
df_train = df_train[df_train["CAMPAÑA"] != "22"].drop(columns = ["SUPERFICIE", "P/S"])

In [201]:
df_pca = df_meteo_eto.drop(columns = ["validTimeUtc", "ID_ESTACION"])
pca = PCA(n_components=0.99)
pdf_fitted = pca.fit(df_pca.values)

In [202]:
df_pca.shape

(51160, 357)

In [203]:
def read_tables(file):
    df_meteo_eto = pd.read_csv(file, index_col = False)
    df_meteo_eto.validTimeUtc = pd.to_datetime(df_meteo_eto.validTimeUtc)
    df_meteo_eto.loc[:, "validTimeUtc"] = df_meteo_eto.loc[:, "validTimeUtc"] + pd.Timedelta(days=185)
    years = df_meteo_eto["validTimeUtc"].dt.year.unique()
    estacions = df_meteo_eto["ID_ESTACION"].unique()
    columns_to_store = df_meteo_eto.columns
    columns_to_store = columns_to_store.drop(["ID_ESTACION", "validTimeUtc"])


    meteo_eto_dict = {}

    for year in years[:-1]:
        for estacion in estacions:
            df_aux = df_meteo_eto.loc[(df_meteo_eto["ID_ESTACION"] == estacion) & (df_meteo_eto["validTimeUtc"].dt.year == year),columns_to_store]
            if year == 2016 or year == 2020:
                df_aux2 = np.array(df_aux.values, dtype=np.float32)[1+240:,:]
            else:
                df_aux2 = np.array(df_aux.values, dtype=np.float32)[240:,:]
            meteo_eto_dict[f"{year}_{estacion}"] = df_aux2

    return meteo_eto_dict

In [204]:
def pca_func(taules, pca_fitted):
    taula_nova = dict()
    for k,v in taules.items():
        taula_nova[k] = pca_fitted.transform(v)
    return taula_nova

In [205]:
fitxer = "../../DATASETS_TRACTATS/df_meteo_eto.csv"
df_year_estacion = read_tables(fitxer)
taula_pca = pca_func(df_year_estacion, pdf_fitted)

In [206]:
df_year_estacion_mostres = "20" + df_train["CAMPAÑA"].astype(str)+ "_" + df_train["ID_ESTACION"].astype(str)

In [207]:
for i in range(1250):
    df_train["c"+str(i)] = 0

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

...

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["c"+str(i)] = 0
C:\Users\adars\AppData\Local\Temp\ipykernel_8780\1067998810.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.c

In [208]:
df_train["cides"] = df_year_estacion_mostres

C:\Users\adars\AppData\Local\Temp\ipykernel_8780\2065210182.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["cides"] = df_year_estacion_mostres


In [209]:
df_train

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,...,c1241,c1242,c1243,c1244,c1245,c1246,c1247,c1248,c1249,cides
0,16,1,1,1,1,1,1,1,1,22413.779,...,0,0,0,0,0,0,0,0,0,2016_1
1,16,2,1,1,1,1,1,1,1,44076.221,...,0,0,0,0,0,0,0,0,0,2016_1
2,16,3,2,2,2,2,2,1,1,16900.000,...,0,0,0,0,0,0,0,0,0,2016_2
3,16,4,3,2,2,3,2,1,1,7320.000,...,0,0,0,0,0,0,0,0,0,2016_2
4,16,5,4,2,2,4,2,1,1,11294.400,...,0,0,0,0,0,0,0,0,0,2016_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6257,21,851,99,16,3,7,1,1,1,28160.100,...,0,0,0,0,0,0,0,0,0,2021_16
6258,21,853,99,16,3,1,1,1,1,41310.000,...,0,0,0,0,0,0,0,0,0,2021_16
6259,21,853,99,16,3,4,1,1,1,45420.000,...,0,0,0,0,0,0,0,0,0,2021_16
6260,21,854,99,16,3,5,1,1,1,56140.000,...,0,0,0,0,0,0,0,0,0,2021_16


In [210]:
for v in df_year_estacion_mostres.unique():
    df_train.loc[df_train["cides"] == v, 'c0':'c1249'] = taula_pca[v].flatten()

In [211]:
df_train.drop(columns = 'cides', inplace = True)

In [212]:
df_train

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,...,c1240,c1241,c1242,c1243,c1244,c1245,c1246,c1247,c1248,c1249
0,16,1,1,1,1,1,1,1,1,22413.779,...,-16001.695326,-1104.301281,1204.210668,87.429587,-329.092076,-484.009851,31.880618,-99.195318,-156.427705,-273.670335
1,16,2,1,1,1,1,1,1,1,44076.221,...,-16001.695326,-1104.301281,1204.210668,87.429587,-329.092076,-484.009851,31.880618,-99.195318,-156.427705,-273.670335
2,16,3,2,2,2,2,2,1,1,16900.000,...,-16001.733910,-1104.364548,1204.171609,87.372538,-328.989938,-483.950813,31.920579,-99.304049,-156.561252,-273.629001
3,16,4,3,2,2,3,2,1,1,7320.000,...,-16001.733910,-1104.364548,1204.171609,87.372538,-328.989938,-483.950813,31.920579,-99.304049,-156.561252,-273.629001
4,16,5,4,2,2,4,2,1,1,11294.400,...,-16001.733910,-1104.364548,1204.171609,87.372538,-328.989938,-483.950813,31.920579,-99.304049,-156.561252,-273.629001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6257,21,851,99,16,3,7,1,1,1,28160.100,...,4018.422471,-4418.716083,-564.617531,-1465.328285,4604.151048,942.145239,517.931036,-1578.032349,812.288204,-325.124977
6258,21,853,99,16,3,1,1,1,1,41310.000,...,4018.422471,-4418.716083,-564.617531,-1465.328285,4604.151048,942.145239,517.931036,-1578.032349,812.288204,-325.124977
6259,21,853,99,16,3,4,1,1,1,45420.000,...,4018.422471,-4418.716083,-564.617531,-1465.328285,4604.151048,942.145239,517.931036,-1578.032349,812.288204,-325.124977
6260,21,854,99,16,3,5,1,1,1,56140.000,...,4018.422471,-4418.716083,-564.617531,-1465.328285,4604.151048,942.145239,517.931036,-1578.032349,812.288204,-325.124977


In [227]:
def grid(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBRegressor(),
                param_grid={"n_estimators": [100],
                            "learning_rate": [0.15],
                            "max_depth": [40],
                            "min_child_weight": [1],
                            "gamma":[0.1],
                            "colsample_bytree":[0.4],
                            "booster": ["dart"]},
                cv=5, scoring=["r2", "neg_root_mean_squared_error"], refit = "neg_root_mean_squared_error", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    
    return gsc

In [231]:
X = df_train.drop(columns = ["PRODUCCION", "CAMPAÑA"])
y = df_train["PRODUCCION"]

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

In [265]:
model = xgb.XGBRegressor(
    n_estimators= 100, 
    learning_rate = 0.15, 
    max_depth = 10, 
    min_child_weight = 1, 
    gamma = 0.1, 
    booster = "gbtree", 
    colsample_bytree=0.8,
    subsamples = 0.6,
    n_jobs = -1)
model.fit(X_train, y_train)

[22:26:00] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "subsamples" } are not used.



XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.15, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [266]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

In [267]:
np.sqrt(mse)

7270.154287920361

In [272]:
model2 = xgb.XGBRegressor(
    n_estimators= 150, 
    learning_rate = 0.15, 
    max_depth = 5, 
    min_child_weight = 1, 
    gamma = 0.1, 
    booster = "dart", 
    colsample_bytree=0.8,
    subsamples = 0.6,
    n_jobs = -1)
model.fit(X_train, y_train)

[22:28:05] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "subsamples" } are not used.



XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.15, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [273]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

In [274]:
np.sqrt(mse)

7270.154287920361

n_estimators: 100  -> 7492
n_estimators: 200 -> 7266.522
n_estimators: 300 -> 7213